In [4]:
from pyspark.sql.functions import explode, col

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
%%sql
SHOW DATABASES;

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1663214425655_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [2]:
df = spark.read.json('s3a://trabajo1-mariana-laura-juan/Transient/Medellin/Datos_SIATA_Vaisala_temperatura.json')
df.printSchema()
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ciudad: string (nullable = true)
 |-- codigo: string (nullable = true)
 |-- datos: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- datos: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- calidad: double (nullable = true)
 |    |    |    |    |-- valor: double (nullable = true)
 |    |    |    |    |-- variableConsulta: string (nullable = true)
 |    |    |-- fecha: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- subcuenca: string (nullable = true)

+----------+------+--------------------+--------+----------+--------------------+------------------+
|    ciudad|codigo|               datos| latitud|  longitud|              nombre|         subcuenca|
+----------+------+--------------------+--------+----------+--------------------+------------------+
|  Medellin|    59|[{[{1.0, 18.7, te..

In [5]:
df1 = df.select(explode(df.datos))
df1.printSchema()
print(df1.count())
df1.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- col: struct (nullable = true)
 |    |-- datos: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- calidad: double (nullable = true)
 |    |    |    |-- valor: double (nullable = true)
 |    |    |    |-- variableConsulta: string (nullable = true)
 |    |-- fecha: string (nullable = true)

561613
+--------------------+
|                 col|
+--------------------+
|{[{1.0, 18.7, tem...|
|{[{1.0, 18.6, tem...|
|{[{1.0, 18.6, tem...|
|{[{1.0, 18.5, tem...|
|{[{1.0, 18.5, tem...|
|{[{1.0, 18.5, tem...|
|{[{1.0, 18.5, tem...|
|{[{1.0, 18.5, tem...|
|{[{1.0, 18.5, tem...|
|{[{1.0, 18.5, tem...|
|{[{1.0, 18.5, tem...|
|{[{1.0, 18.5, tem...|
|{[{1.0, 18.5, tem...|
|{[{1.0, 18.5, tem...|
|{[{1.0, 18.4, tem...|
|{[{1.0, 18.5, tem...|
|{[{1.0, 18.4, tem...|
|{[{1.0, 18.5, tem...|
|{[{1.0, 18.5, tem...|
|{[{1.0, 18.5, tem...|
+--------------------+
only showing top 20 rows

In [6]:
df2 = df1.rdd.map(lambda row: (row.col["fecha"],row.col["datos"][0]["variableConsulta"],row.col["datos"][0]["calidad"],row.col["datos"][0]["valor"])).toDF(["fecha","variableConsulta","calidad","valor"])
df2.printSchema()
df2.show()
print(df2.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- fecha: string (nullable = true)
 |-- variableConsulta: string (nullable = true)
 |-- calidad: double (nullable = true)
 |-- valor: double (nullable = true)

+-------------------+----------------+-------+-----+
|              fecha|variableConsulta|calidad|valor|
+-------------------+----------------+-------+-----+
|2022-06-07 18:15:00|     temperatura|    1.0| 18.7|
|2022-06-07 18:16:00|     temperatura|    1.0| 18.6|
|2022-06-07 18:17:00|     temperatura|    1.0| 18.6|
|2022-06-07 18:18:00|     temperatura|    1.0| 18.5|
|2022-06-07 18:19:00|     temperatura|    1.0| 18.5|
|2022-06-07 18:20:00|     temperatura|    1.0| 18.5|
|2022-06-07 18:21:00|     temperatura|    1.0| 18.5|
|2022-06-07 18:22:00|     temperatura|    1.0| 18.5|
|2022-06-07 18:23:00|     temperatura|    1.0| 18.5|
|2022-06-07 18:24:00|     temperatura|    1.0| 18.5|
|2022-06-07 18:25:00|     temperatura|    1.0| 18.5|
|2022-06-07 18:26:00|     temperatura|    1.0| 18.5|
|2022-06-07 18:27:00|     temperatura|

In [8]:
df2.write.option("header",True).csv('s3a://trabajo1-mariana-laura-juan/Raw/Medellin/Medellin_temperatura')
print('Saved to Raw Zone')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Saved to Raw Zone